In [ ]:
# imports
import random
import numpy as np

In [ ]:
rewards = {" ":0.0, "D":-1.0, "J":1.0, "S":0.0}

def init_space():
    space = [
        ['S', ' ', ' ', ' '],
        ['D', ' ', 'D', ' '],
        [' ', ' ', ' ', 'D'],
        [' ', 'D', ' ', 'J'],
        ]
    return space;


def isWin (space, position):
    (x,y) = position

    if(space[x][y]== 'J'):
        return True

def application_action(action, position, space):

    reward = 0.0
    
    (x, y) = position

    fin = space[x][y]== 'J'
    nextPos = position
    #haut
    if action == 0:
        nextPos = (x,y+1)
    #droit
    elif action == 1:
        nextPos = (x+1,y)
    #bas
    elif action == 2:
        nextPos = (x,y-1);
    #gauche
    elif action == 3:
        nextPos = (x-1,y);

    if (nextPos[0] < len(space) and nextPos[1] < len(space) and nextPos[0] >=0 and nextPos[1] >=0 ):
        print(nextPos[0])
        print(nextPos[1])
        print(len(space))

        position = nextPos

    return position, reward, fin

In [ ]:
space = init_space();
#position de l'agent
player_pos = (0, 0)
nbtries = 20

(player_pos,reward,fin) = application_action(3,player_pos,space)


print(space)
print(player_pos)